In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import sys

In [ ]:
# sample source 
m = 3000
dim = 2
seed = 3
n = m
#ss = data.SSGaussMeanDiff(dim, my=1)
ss = data.SSGaussVarDiff(dim)
#ss = data.SSBlobs()
tst_data = ss.sample(m, seed=seed)
tr, te = tst_data.split_tr_te(tr_proportion=0.5, seed=10)
#te = tst_data

## mean embedding test

In [ ]:
J = 5
alpha = 0.01
met  = tst.MeanEmbeddingTest.create_fit_gauss_heuristic(te, J, alpha, seed=seed+1)
met.perform_test(te)

## grid search to choose the best Gaussian width

In [ ]:
T = tst.MeanEmbeddingTest.init_locs_2randn(tr, J, seed=seed+1)
mean_sd = tr.mean_std()
scales = 2.0**np.linspace(-5, 5, 50)
list_gwidth = mean_sd*scales
besti, powers = tst.MeanEmbeddingTest.grid_search_gwidth(tr, T, list_gwidth, alpha)

# plot
plt.plot(list_gwidth, powers, 'o-')
plt.xscale('log', basex=2)
plt.xlabel('Gaussian width')
plt.ylabel('Test power')
plt.title('Mean std: %.3g. Best chosen: %.2g'%(mean_sd, list_gwidth[besti]) )

In [ ]:
# test with the best Gaussian with 
best_width = list_gwidth[besti]
met_grid = tst.MeanEmbeddingTest(T, best_width, alpha)
met_grid.perform_test(te)